In [16]:
import pandas as pd
from tqdm import tqdm
from nixtla import NixtlaClient

In [17]:
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-WgE8MEuLXfee5pzkScwGhD0FkGhMcM2TF9HlmyBdsYqkRdznaQHgMyd1t4PIWfo8TLhZkF9NRKm65GKL' # misaki api
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [18]:
start_point = 200*12
batch_size = 12
num = 800
output_file = './data/forecasts_201_1000.csv'

flatten_df = pd.read_csv('./data/flattened_loop_sensor_test_12h.csv')
flatten_df = flatten_df.iloc[start_point :start_point + batch_size * num]
flatten_df['t_1h'] = pd.to_datetime(flatten_df['t_1h'])
print(flatten_df.shape)
flatten_df.head()

(9600, 5)


,id,iu_ac,t_1h,etat_barre,q
2400,201,5,2023-09-06 05:00:00,3,924.0
2401,201,5,2023-09-06 06:00:00,3,1364.0
2402,201,5,2023-09-06 07:00:00,3,1530.0
2403,201,5,2023-09-06 08:00:00,3,1568.0
2404,201,5,2023-09-06 09:00:00,3,1450.0


In [19]:
num_batches = len(flatten_df) // batch_size
all_forecasts = []

In [20]:
for i in tqdm(range(num_batches), desc='Forecasting'):
    # 读取12条数据
    df = flatten_df.iloc[i * batch_size:(i + 1) * batch_size]
    # 进行预测
    timegpt_fcst_df = nixtla_client.forecast(
        df=df, 
        h=1,  # 预测步长
        # finetune_steps=10,
        # finetune_loss='mae',   
        freq='h', 
        time_col='t_1h', 
        target_col='q', 
        model='timegpt-1'
    )
    print(timegpt_fcst_df.head())
    all_forecasts.append(timegpt_fcst_df)

# 将所有预测结果合并为一个DataFrame
all_forecasts_df = pd.concat(all_forecasts, ignore_index=False)
print(all_forecasts_df.shape)

all_forecasts_df.head()

Forecasting:   0%|          | 0/800 [00:00<?, ?it/s]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 1/800 [00:04<58:57,  4.43s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...


                  t_1h      TimeGPT
0  2023-09-06 17:00:00  1631.873047


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 2/800 [00:08<52:46,  3.97s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...


                  t_1h      TimeGPT
0  2023-09-07 22:00:00  1463.474121


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 3/800 [00:12<52:33,  3.96s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...


...

INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting: 100%|█████████▉| 799/800 [52:20<00:04,  4.60s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...


                  t_1h      TimeGPT
0  2023-04-20 18:00:00  1787.464233


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting: 100%|██████████| 800/800 [52:24<00:00,  3.93s/it]

                  t_1h      TimeGPT
0  2023-04-21 23:00:00  1351.160034
(800, 2)


,t_1h,TimeGPT
0,2023-09-06 17:00:00,1631.873047
0,2023-09-07 22:00:00,1463.474121
0,2023-09-09 03:00:00,1010.350220
0,2023-09-10 08:00:00,400.918945
0,2023-09-11 13:00:00,1595.089233


In [23]:
# Reset the index to start from 1
all_forecasts_df.reset_index(drop=True, inplace=True)
all_forecasts_df.index += 1 + int(start_point/12)

# Save the DataFrame to a CSV file with the new index
all_forecasts_df.to_csv(output_file, index=True, index_label='id')